<a href="https://colab.research.google.com/github/AkankshaB123/leet-code/blob/main/Hands-on%3AContinuous_13w_inactivity_ipynb(16_12).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 12) Continuous 13 weeks inactivity Calculation
# Retail Year, Retail week , Seller , GMV, sellers migrated week
# 2025,25,142525,1000, 3
# 2025,26,142525,1200,3

# Check in Next 26 weeks whether seller has atleast 13 weeks of continuous inactivity until end of period

In [34]:
import pandas as pd
from datetime import date, timedelta

rows = []
seller_id = 142525

# Monday anchor – start inactivity
start_week = date(2025, 1, 6)  # Monday

# 13 continuous inactive weeks
for i in range(13):
    rows.append({
        "seller_id": seller_id,
        "dt": start_week + timedelta(weeks=i),
        "gmv": 0
    })

# Reactivation exactly 2 weeks after inactivity end
rows.append({
    "seller_id": seller_id,
    "dt": start_week + timedelta(weeks=14),  # 2025-04-14
    "gmv": 500
})

# Retail week 25 of 2025
rows.append({
    "seller_id": seller_id,
    "dt": date(2025, 6, 16),
    "gmv": 1000
})

# Retail week 26 of 2025
rows.append({
    "seller_id": seller_id,
    "dt": date(2025, 6, 23),
    "gmv": 1200
})

revised_txn_data = pd.DataFrame(rows)
revised_txn_data

,seller_id,dt,gmv
0,142525,2025-01-06,0
1,142525,2025-01-13,0
2,142525,2025-01-20,0
3,142525,2025-01-27,0
4,142525,2025-02-03,0
5,142525,2025-02-10,0
6,142525,2025-02-17,0
7,142525,2025-02-24,0
8,142525,2025-03-03,0
9,142525,2025-03-10,0


In [4]:
import duckdb
import pandas as pd
from datetime import date, timedelta

In [5]:
con = duckdb.connect()
con.register("txn_data", txn_data)

In [46]:
query = """
WITH txn_data AS (
  SELECT
    seller_id AS user_id,
    DATE_TRUNC('year', dt) AS year_date,
    DATE_TRUNC('week', dt) AS week_date,
    EXTRACT('week' FROM dt) AS retail_week,
    SUM(gmv) AS weekly_gmv
  FROM revised_txn_data
  GROUP BY 1,2,3,4
),

inactivity_flag_added AS (
  SELECT *,
         CASE WHEN weekly_gmv = 0 THEN 1 ELSE 0 END AS inactivity_flag
  FROM txn_data
),

rolling_13_tbl AS (
  SELECT *,
         SUM(inactivity_flag) OVER (
           PARTITION BY user_id
           ORDER BY week_date
           ROWS BETWEEN 12 PRECEDING AND CURRENT ROW
         ) AS rolling_13
  FROM inactivity_flag_added
),

first_inactivity_week AS (
  SELECT
    user_id,
    MIN(week_date) AS first_inactivity_week_date
  FROM rolling_13_tbl
  WHERE rolling_13 = 13
  GROUP BY user_id
),

first_reactivation_per_seller AS (
    SELECT
      r.user_id,
      MIN(r.week_date) AS migrated_first_week_date
    FROM txn_data r
    JOIN first_inactivity_week i
      ON r.user_id = i.user_id
  WHERE weekly_gmv > 0
    AND week_date > i.first_inactivity_week_date
  GROUP BY r.user_id
)

SELECT
  EXTRACT(year FROM r.week_date) AS retail_year,
  EXTRACT(week FROM r.week_date) AS retail_week,
  r.user_id AS seller,
  r.weekly_gmv AS gmv,
  DATE_DIFF(
    'week',
    i.first_inactivity_week_date,
    f.migrated_first_week_date
  ) + 1 AS sellers_migrated_week
FROM txn_data r
JOIN first_inactivity_week i
  ON r.user_id = i.user_id
JOIN first_reactivation_per_seller f
  ON r.user_id = f.user_id
WHERE r.week_date > f.migrated_first_week_date  --start from migrated week
  AND r.week_date < f.migrated_first_week_date + INTERVAL '26 weeks'
"""
result = con.execute(query).fetchdf()
result

,retail_year,retail_week,seller,gmv,sellers_migrated_week
0,2025,25,142525,1000.0,3
1,2025,26,142525,1200.0,3
